In [88]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from hipe4ml.model_handler import ModelHandler
from hipe4ml.tree_handler import TreeHandler
from hipe4ml.analysis_utils import train_test_generator
from hipe4ml import plot_utils, analysis_utils

In [89]:
import uproot

In [ ]:
def hist_to_df(hist):
    """
    Convert a TH1 or TH2 histogram to a pandas DataFrame.
    """
    if hist.classname.startswith("TH1"):
        values, edges = hist.to_numpy()
        # x_axis = hist.axis()
        x_label = hist.name or "x"  # use axis label if available
        return pd.DataFrame({
            x_label + "_left": edges[:-1],
            x_label + "_right": edges[1:],
            "counts": values
        })
    elif hist.classname.startswith("TH2"):
        values, (xedges, yedges) = hist.to_numpy()
        x_label = hist.name or "x"
        y_label = hist.name or "y"
        X, Y = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")
        return pd.DataFrame({
            x_label: X.ravel(),
            y_label: Y.ravel(),
            "counts": values.ravel()
        })
    else:
        raise NotImplementedError("Only TH1 and TH2 supported")
    
def rebin_df(df, new_edges, x_left="x_left", x_right="x_right", counts="counts"):
    new_counts = []
    
    for i in range(len(new_edges)-1):
        bin_left = new_edges[i]
        bin_right = new_edges[i+1]
        # Select all original bins that overlap the new bin
        mask = (df[x_right] > bin_left) & (df[x_left] < bin_right)
        
        # Sum the counts of overlapping bins (simple sum; more precise weighting can be applied if needed)
        new_counts.append(df.loc[mask, counts].sum())
        
    return pd.DataFrame({
        "bin_left": new_edges[:-1],
        "bin_right": new_edges[1:],
        "counts": new_counts
    })

import os
import ROOT

# def df_to_root_th1(df, hist_name="hist", hist_title="Histogram"):
#     """
#     Convert a pandas DataFrame with 'bin_left', 'bin_right', 'counts' to a TH1
#     """

#     bin_edges = np.append(df["bin_left"].values, df["bin_right"].values[-1])
#     counts = df["counts"].values
#     nBins = len(counts)
#     hist = ROOT.TH1F(hist_name, hist_title, nBins, bin_edges)
#     for i, c in enumerate(counts):
#         hist.SetBinContent(i+1, c)  # ROOT bins start at 1
#     return hist


def df_to_root_graph(df, graph_name="graph", graph_title="Histogram with errors"):
    """
    Convert a pandas DataFrame with 'bin_left', 'bin_right', 'counts' to a ROOT TGraphErrors
    with horizontal error bars corresponding to bin widths.
    """
    bin_left = df["bin_left"].values
    bin_right = df["bin_right"].values
    counts = df["counts"].values

    x = (bin_left + bin_right) / 2.0           # bin centers
    y = counts
    ex = (bin_right - bin_left) / 2.0          # horizontal errors = half bin width
    # ey = np.sqrt(counts)                        # vertical errors (Poisson), can set to 0 if not needed
    ey=np.zeros_like(y)  # No vertical errors, can be set to zero
    graph = ROOT.TGraphErrors(len(x),
                              x.astype(float), y.astype(float),
                              ex.astype(float), ey.astype(float))
    graph.SetName(graph_name)
    graph.SetTitle(graph_title)
    graph.SetMarkerStyle(20)
    # graph.SetMarkerSize(0.8)

    return graph

# LEGEND_SIZE_DEFAULT = 0.02
# LEGEND_COORDS_DEFAULT = [0.2, 0.6, 0.4, 0.9]
# def createLegend(legLines, legCoords=LEGEND_COORDS_DEFAULT, textSize=LEGEND_SIZE_DEFAULT, isMC=False):
#     legend = TLegend(*legCoords)
#     legend.SetFillStyle(0)
#     legend.SetBorderSize(0)
#     legend.SetTextSize(textSize)
#     for legLine in legLines:
#         legend.AddEntry(legend, legLine, "")
#     legend.AddEntry(legend, "Data, |y| < 0.9", "")
#     if isMC:
#         legend.AddEntry(legend, "MC, |y| < 0.9", "")
#     return legend

In [91]:
dir = '~/alice/O2Physics/PWGJE/Tasks/JPsiWorkDir/JPsiMC/DQEfficiency'
filename = 'AnalysisResultsMCGen25-08-14.root'
path = f"{dir}/{filename}"
with uproot.open(path) as root_file:
    print(root_file.keys())
    analysis_dir = root_file['analysis-same-event-pairing']
print(analysis_dir.keys()) 
output_dir = analysis_dir['output']

['analysis-event-selection;1', 'analysis-event-selection/output;1', 'analysis-same-event-pairing;1', 'analysis-same-event-pairing/output;1']
['output;1']


In [92]:
for JPsiList in output_dir:
    print("JPsiList: ", JPsiList.member("fName"))
    if JPsiList.member("fName") == "MCTruthGen_promptJpsi":
        genPJPsiList = JPsiList
    elif JPsiList.member("fName") == "PairsBarrelSEPM_jpsiO2MCdebugCuts14":
        recoPJPsiList = JPsiList

JPsiList:  PairsBarrelSEPM_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEPP_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEMM_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEPM_ambiguousInBunch_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEPP_ambiguousInBunch_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEMM_ambiguousInBunch_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEPM_ambiguousOutOfBunch_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEPP_ambiguousOutOfBunch_jpsiO2MCdebugCuts14
JPsiList:  PairsBarrelSEMM_ambiguousOutOfBunch_jpsiO2MCdebugCuts14
JPsiList:  MCTruthGen_promptJpsi
JPsiList:  MCTruthGenSel_promptJpsi
JPsiList:  MCTruthGen_nonPromptJpsi
JPsiList:  MCTruthGenSel_nonPromptJpsi


In [93]:
print("genPJPsiList hists:",)
for hist in genPJPsiList:
    print(hist.member("fName"))
    if hist.member("fName") == "PtMC":
        genPJPsiptDf = hist_to_df(hist)


print("\nrecoPJPsiList hists:",)
for hist in recoPJPsiList:
    print(hist.member("fName"))
    if hist.member("fName") == "Pt":
        recoPJPsiptDf = hist_to_df(hist)

genPJPsiList hists:
PtMC
EtaMC
PhiMC
YMC
CentFT0CMC
PtMC_YMC
EtaMC_PtMC
VzMC
VzMC_VtxZMC
Weight
MCImpPar_CentFT0CMC

recoPJPsiList hists:
Mass
Mass_HighRange
Pt
Mass_Pt
Mass_PtFine
Eta_Pt
Y_Pt
Mass_VtxZ
UsedKF
Lz
Lxy
Lxyz
Tauz
Tauxy
Tauxy_Mass_Pt
Tauz_Mass_Pt
LzProj
LxyProj
LxyzProj
TauzProj
TauxyProj
TauxyProj_Mass_Pt
TauzProj_Mass_Pt
TauxyzProj
LxyProj_Pt
LxyProj_Mass_Pt
LzProj_Mass_Pt
CosPointingAngle
VtxingChi2PCA


In [94]:
print("genPJPsiptDf: ")
genPJPsiptDf

genPJPsiptDf: 


,PtMC_left,PtMC_right,counts
0,0.0,0.1,41.0
1,0.1,0.2,136.0
2,0.2,0.3,223.0
3,0.3,0.4,294.0
4,0.4,0.5,337.0
...,...,...,...
195,19.5,19.6,0.0
196,19.6,19.7,2.0
197,19.7,19.8,1.0
198,19.8,19.9,0.0


In [95]:
print("recoPJPsiptDf: ")
recoPJPsiptDf

recoPJPsiptDf: 


,Pt_left,Pt_right,counts
0,0.00,0.01,0.0
1,0.01,0.02,0.0
2,0.02,0.03,0.0
3,0.03,0.04,0.0
4,0.04,0.05,0.0
...,...,...,...
1995,19.95,19.96,0.0
1996,19.96,19.97,0.0
1997,19.97,19.98,0.0
1998,19.98,19.99,0.0


In [144]:
ptBins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 12]  # De

genPJPsiptDfReb = rebin_df(genPJPsiptDf, ptBins, x_left="PtMC_left", x_right="PtMC_right", counts="counts")
recoPJPsiptDfReb = rebin_df(recoPJPsiptDf, ptBins, x_left="Pt_left", x_right="Pt_right", counts="counts")

In [145]:
print("genPJPsiptDfReb: ")
genPJPsiptDfReb

genPJPsiptDfReb: 


,bin_left,bin_right,counts
0,0,1,3708.0
1,1,2,7325.0
2,2,3,6483.0
3,3,4,4763.0
4,4,5,3076.0
5,5,6,1823.0
6,6,7,1070.0
7,7,8,600.0
8,8,12,791.0


In [146]:
print("recoPJPsiptDfReb: ")
recoPJPsiptDfReb

recoPJPsiptDfReb: 


,bin_left,bin_right,counts
0,0,1,218.0
1,1,2,296.0
2,2,3,324.0
3,3,4,273.0
4,4,5,227.0
5,5,6,134.0
6,6,7,83.0
7,7,8,33.0
8,8,12,28.0


In [147]:
df_eff = recoPJPsiptDfReb.copy()
df_eff['counts'] = df_eff['counts'] / genPJPsiptDfReb['counts']
df_eff

,bin_left,bin_right,counts
0,0,1,0.058792
1,1,2,0.040410
2,2,3,0.049977
3,3,4,0.057317
4,4,5,0.073797
5,5,6,0.073505
6,6,7,0.077570
7,7,8,0.055000
8,8,12,0.035398


In [148]:
# %jsroot on
# eff_hist = df_to_root_th1(df_eff,
#                hist_name="efficiency_hist",
#                hist_title="Efficiency Histogram")

# c = ROOT.TCanvas()
# # eff_hist.SetLineColor(ROOT.kBlue-2)  
# eff_hist.GetXaxis().SetTitle("p_{T} (GeV/c)")
# eff_hist.GetYaxis().SetTitle("Efficiency")
# eff_hist.SetMarkerStyle(20)
# # eff_hist.SetMarkerSize(0.5)
# eff_hist.SetMarkerColor(ROOT.kBlue-2)


# eff_hist.Draw("P")

# legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
# legend.AddEntry(eff_hist, "Efficiency")
# legend.Draw()

# c.Draw()

# Save to ROOT file
# root_file = ROOT.TFile("Efficiency.root", "RECREATE")
# hist.Write()
# root_file.Close()

In [153]:
graph = df_to_root_graph(df_eff, graph_name="efficiency_graph", graph_title="Efficiency vs pT")
c = ROOT.TCanvas()
graph.Draw("AP")  # A = axis, P = points with error bars
graph.GetXaxis().SetTitle("p_{T} (GeV/c)")
graph.GetYaxis().SetTitle("A x #epsilon")
graph.SetMarkerColor(ROOT.kRed-2)
graph.SetLineColor(ROOT.kRed-2)
graph.SetLineWidth(2)

legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legend.AddEntry(legend,"pp #sqrt s = 13.6 TeV", "")
legend.AddEntry(graph, "Efficiency", "p")
# legend.AddEntry(graph2, "Dataset 2", "p")
legend.SetFillStyle(0)
legend.SetBorderSize(0)
legend.SetTextSize(0.04)
legend.Draw()

c.Draw()

In [ ]:
# recoFile = "~/alice/O2Physics/PWGJE/Tasks/JPsiWorkDir/JPsiMC/DQEfficiency/AnalysisResults_trees.root"
# genFile = "~/alice/JPsiDatasets/MC/RawMC/AO2DMC_LHC25b16_0_550369_AOD_001.root"

# f_reco = uproot.open(recoFile)
# f_gen = uproot.open(genFile)

# dfs = [] # Needed if the root files contain multiple TDirectoryFiles (representing dataframes) with TTrees

# for key in f_reco.keys():
#     name = key.decode() if isinstance(key, bytes) else key  # ROOT 6.x returns bytes
#     if name.startswith("DF"):
#         print(f"Processing directory: {name}")
#         tree_path = f"{name}/O2rtdielectron" # Without ";1"
#         try:
#             th = TreeHandler(recoFile, tree_name=tree_path) # Returns a pd.DataFrame()
#             dfs.append(th.data)
#         except Exception as e:
#             print(f"  Could not load {tree_path}: {e}")

# # Concatenar todos os DataFrames
# full_data = pd.concat(dfs, ignore_index=True)
# print("Total entries:", len(full_data))

# # Exemplo: aplicar cut de massa e calcular eficiência global
# mask = (full_data['mass'] > 3.0) & (full_data['mass'] < 3.2)
# n_total = len(full_data)
# n_pass  = mask.sum()
# efficiency = n_pass / n_total
# print("Global efficiency:", efficiency)

In [ ]:
recoH = TreeHandler("~/alice/O2Physics/PWGJE/Tasks/JPsiWorkDir/JPsiMC/DQEfficiency/AnalysisResults_trees.root")
# =3 is prompt jpsi and =5 is nonprpmpt jpsi
promptH = dataH.get_subset('fMcDecision==3')
nonpromptH = dataH.get_subset('fMcDecision==5')

In [ ]:
train_test_data = train_test_generator([promptH, nonpromptH], [1,0], test_size=0.5, random_state=42)

In [ ]:
vars_to_draw = promptH.get_var_names()
vars_to_draw.remove('fMass')
vars_to_draw.remove('fPt')
vars_to_draw.remove('fEta')
vars_to_draw.remove('fPhi')
vars_to_draw.remove('fSign')
vars_to_draw.remove('fFilterMap')
vars_to_draw.remove('fMcDecision')
vars_to_draw.remove('fPt1')
vars_to_draw.remove('fEta1')
vars_to_draw.remove('fPhi1')
vars_to_draw.remove('fITSClusterMap1')
vars_to_draw.remove('fITSChi2NCl1')
vars_to_draw.remove('fTPCNClsCR1')
vars_to_draw.remove('fTPCNClsFound1')
vars_to_draw.remove('fTPCChi2NCl1')
# vars_to_draw.remove('fDcaXY1')
# vars_to_draw.remove('fDcaZ1')
vars_to_draw.remove('fTPCSignal1')
vars_to_draw.remove('fTPCNSigmaEl1')
vars_to_draw.remove('fTPCNSigmaPi1')
vars_to_draw.remove('fTPCNSigmaPr1')
vars_to_draw.remove('fTOFBeta1')
vars_to_draw.remove('fTOFNSigmaEl1')
vars_to_draw.remove('fTOFNSigmaPi1')
vars_to_draw.remove('fTOFNSigmaPr1')
vars_to_draw.remove('fPt2')
vars_to_draw.remove('fEta2')
vars_to_draw.remove('fPhi2')
vars_to_draw.remove('fITSClusterMap2')
vars_to_draw.remove('fITSChi2NCl2')
vars_to_draw.remove('fTPCNClsCR2')
vars_to_draw.remove('fTPCNClsFound2')
vars_to_draw.remove('fTPCChi2NCl2')
# vars_to_draw.remove('fDcaXY2')
# vars_to_draw.remove('fDcaZ2')
vars_to_draw.remove('fTPCSignal2')
vars_to_draw.remove('fTPCNSigmaEl2')
vars_to_draw.remove('fTPCNSigmaPi2')
vars_to_draw.remove('fTPCNSigmaPr2')
vars_to_draw.remove('fTOFBeta2')
vars_to_draw.remove('fTOFNSigmaEl2')
vars_to_draw.remove('fTOFNSigmaPi2')
vars_to_draw.remove('fTOFNSigmaPr2')
# vars_to_draw.remove('fDCAxyzTrk0KF')
# vars_to_draw.remove('fDCAxyzTrk1KF')
# vars_to_draw.remove('fDCAxyzBetweenTrksKF')
# vars_to_draw.remove('fDCAxyTrk0KF')
# vars_to_draw.remove('fDCAxyTrk1KF')
# vars_to_draw.remove('fDCAxyBetweenTrksKF')
# vars_to_draw.remove('fDeviationTrk0KF')
# vars_to_draw.remove('fDeviationTrk1KF')
# vars_to_draw.remove('fDeviationxyTrk0KF')
# vars_to_draw.remove('fDeviationxyTrk1KF')
vars_to_draw.remove('fMassKFGeo')
# vars_to_draw.remove('fChi2OverNDFKFGeo')
# vars_to_draw.remove('fDecayLengthKFGeo')
# vars_to_draw.remove('fDecayLengthOverErrKFGeo')
# vars_to_draw.remove('fDecayLengthXYKFGeo')
# vars_to_draw.remove('fDecayLengthXYOverErrKFGeo')
# vars_to_draw.remove('fPseudoproperDecayTimeKFGeo')
# vars_to_draw.remove('fPseudoproperDecayTimeErrKFGeo')
# vars_to_draw.remove('fCosPAKFGeo')
# vars_to_draw.remove('fPairDCAxyz')
# vars_to_draw.remove('fPairDCAxy')
# vars_to_draw.remove('fDeviationPairKF')
# vars_to_draw.remove('fDeviationxyPairKF')
vars_to_draw.remove('fMassKFGeoTop')
# vars_to_draw.remove('fChi2OverNDFKFGeoTop')

In [ ]:
leg_labels = [r'$Non-prompt J/\psi$', r'$Prompt J/\psi$']
plt.rcParams['axes.titlesize'] = 8
plt.rcParams['xtick.labelsize'] = 5
plt.rcParams['ytick.labelsize'] = 5
plt.legend(loc='lower right')

In [ ]:
plot_utils.plot_distr([nonpromptH, promptH], vars_to_draw, bins=100, labels=leg_labels, log=True, density=True, figsize=(12, 7), alpha=0.3, grid=False)
plt.subplots_adjust(left=0.06, bottom=0.06, right=0.99, top=0.96, hspace=0.55, wspace=0.55)
plt.savefig('figs/distribution.pdf')

In [ ]:
plot_utils.plot_corr([nonpromptH, promptH], vars_to_draw, leg_labels)
corr1 = plt.figure(2)
corr1.savefig('figs/corr1.pdf')
corr2 = plt.figure(3)
corr1.savefig('figs/corr2.pdf')
print("finish read the data!")

In [ ]:
features_for_train = vars_to_draw.copy()
print("features_for_train:", features_for_train)

In [ ]:
print("test")
model_clf = xgb.XGBClassifier()
model_hdl = ModelHandler(model_clf, features_for_train)
#  run cross-validation trials and pick the best hyperparameters. "n_jobs=-1" means use all available CPU cores
hyper_pars_ranges = {'n_estimators': (200, 1000), 'max_depth': (
    2, 4), 'learning_rate': (0.01, 0.1)}
# default: model_hdl.optimize_params_optuna(train_test_data, hyper_pars_ranges, cross_val_scoring='roc_auc', timeout=120,
                                # n_jobs=-1, n_trials=100, direction='maximize')
print("Starting parameters optmization with Optuna")
model_hdl.optimize_params_optuna(train_test_data, hyper_pars_ranges, cross_val_scoring='roc_auc', timeout=120,
                                n_jobs=-1, n_trials=3, direction='maximize')
print("Parameters optimization with Optuna finished")
#model_hdl.optimize_params_optuna(train_test_data, hyper_pars_ranges, cross_val_scoring='roc_auc', timeout=5,
#                                n_jobs=1, n_trials=3, direction='maximize')

In [ ]:
model_hdl.train_test_model(train_test_data)

In [ ]:
y_pred_train = model_hdl.predict(train_test_data[0], False)
y_pred_test = model_hdl.predict(train_test_data[2], False)

In [ ]:
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams["figure.figsize"] = (20, 15)

In [ ]:
ml_out_fig = plot_utils.plot_output_train_test(model_hdl, train_test_data, 100,
                                               False, leg_labels, True, density=True)
plt.savefig('figs/ml_out_fig.pdf')

In [ ]:
roc_train_test_fig = plot_utils.plot_roc_train_test(train_test_data[3], y_pred_test,
                                                    train_test_data[1], y_pred_train, None, leg_labels)
# plt.savefig('figs/roc.pdf')
roc_train_test_fig.savefig('figs/roc_train_test.pdf')

In [ ]:
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

features_importance = plot_utils.plot_feature_imp(train_test_data, leg_labels, model_hdl, features_for_train,10000, 1)
plt.savefig("figs/features_importance.pdf")
print("finish training")

In [ ]:
for i, fig in enumerate(features_importance):
    fig.savefig(f"figs/features_importance{i}.pdf")

In [ ]:
dataH.apply_model_handler(model_hdl, False)

selected_data_hndl = dataH.get_subset('model_output>0.80')
labels_list = ["after selection","before selection"]
colors_list = ['orangered', 'cornflowerblue']
plot_utils.plot_distr([selected_data_hndl, dataH], column=vars_to_draw.copy(), bins=200, labels=labels_list, colors=colors_list, density=True,fill=True, histtype='step', alpha=0.5)
ax = plt.gca()
ax.set_xlabel(r'm(e^{+}e^{-}) (GeV/$c^2$)')
ax.margins(x=0)
ax.xaxis.set_label_coords(0.9, -0.075)
plt.savefig("figs/distributions_data.pdf")

In [ ]:
# PRECISION_RECALL_PLOT = plot_utils.plot_precision_recall(DATA[3], Y_PRED)
# BDT_EFFICIENCY_PLOT = plot_utils.plot_bdt_eff(THRESHOLD, EFFICIENCY)

# plt.show()
print("finish training")